In [1]:
import pandas as pd
import json
import re
import sqlite3
import os

## Orders data

In [25]:
orders_df = pd.read_csv(r"C:\Users\merug\Downloads\orders.csv")
orders_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


## Users Data

In [26]:
users_df = pd.read_json(r"C:\Users\merug\Downloads\users.json")
users_df

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold
...,...,...,...,...
2995,2996,User_2996,Hyderabad,Gold
2996,2997,User_2997,Hyderabad,Regular
2997,2998,User_2998,Bangalore,Regular
2998,2999,User_2999,Pune,Regular


## Restaurants Data

In [27]:
def load_restaurants_sql(filepath):
    
    try:
        
        with open(filepath, 'r') as f:
            sql_content = f.read()
        
        
        create_pattern = r'CREATE TABLE restaurants \((.*?)\);'
        create_match = re.search(create_pattern, sql_content, re.DOTALL)
        
        if create_match:
            columns_text = create_match.group(1)
            
            columns = [col.strip().split()[0] for col in columns_text.split(',')]
        
        insert_pattern = r"INSERT INTO restaurants VALUES \((.*?)\);"
        matches = re.findall(insert_pattern, sql_content)
        
        
        data = []
        for match in matches:
            
            values = []
            current_value = ""
            in_quotes = False
            
            for char in match + ',':
                if char == "'" and not in_quotes:
                    in_quotes = True
                elif char == "'" and in_quotes:
                    in_quotes = False
                elif char == ',' and not in_quotes:
                    values.append(current_value.strip().strip("'"))
                    current_value = ""
                else:
                    current_value += char
            
            
            parsed_row = [
                int(values[0]),          
                values[1],               
                values[2],               
                float(values[3])         
            ]
            data.append(parsed_row)
        
        # Create DataFrame
        restaurants_df = pd.DataFrame(data, columns=columns)
        
        return restaurants_df
    
    except Exception as e:
        print(f"✗ Error loading SQL file: {str(e)}")
        raise

# Load the restaurants SQL
restaurants_df = load_restaurants_sql(r"C:\Users\merug\Downloads\restaurants.sql")

In [28]:
restaurants_df

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8
...,...,...,...,...
495,496,Restaurant_496,Indian,3.1
496,497,Restaurant_497,Mexican,4.4
497,498,Restaurant_498,Chinese,3.9
498,499,Restaurant_499,Mexican,4.9


## Merge the Datasets

In [29]:
merged_df = pd.merge(orders_df, users_df, on='user_id', how='left', suffixes=('', '_restaurant'))
merged_df

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [30]:
final_dataset = pd.merge(merged_df, restaurants_df, on='restaurant_id', how='left')
final_dataset

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold,Restaurant_492,Italian,4.0


In [38]:
final_dataset.rename(columns={"restaurant_name_x": "restaurant_name", "restaurant_name_y": "restaurant_name_sql"}, inplace=True)
final_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership,restaurant_name_sql,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


### columns order

In [43]:
column_order = [
        'order_id',
        'order_date',
        'total_amount',
    
        'user_id',
        'name',
        'city',
        'membership',

        'restaurant_id',
        'restaurant_name',
        'restaurant_name_sql',
        'cuisine',
        'rating'
    
]

In [46]:
final_dataset = final_dataset[column_order]
final_dataset

,order_id,order_date,total_amount,user_id,name,city,membership,restaurant_id,restaurant_name,restaurant_name_sql,cuisine,rating
0,1,18-02-2023,842.97,2508,User_2508,Hyderabad,Regular,450,New Foods Chinese,Restaurant_450,Mexican,3.2
1,2,18-01-2023,546.68,2693,User_2693,Pune,Regular,309,Ruchi Curry House Multicuisine,Restaurant_309,Indian,4.5
2,3,15-07-2023,163.93,2084,User_2084,Chennai,Gold,107,Spice Kitchen Punjabi,Restaurant_107,Mexican,4.0
3,4,04-10-2023,1155.97,319,User_319,Bangalore,Gold,224,Darbar Kitchen Non-Veg,Restaurant_224,Chinese,4.8
4,5,25-12-2023,1321.91,1064,User_1064,Pune,Regular,293,Royal Eatery South Indian,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,21-05-2023,1211.96,2528,User_2528,Hyderabad,Gold,249,Royal Kitchen North Indian,Restaurant_249,Italian,4.7
9996,9997,06-08-2023,1188.05,2867,User_2867,Bangalore,Regular,267,Darbar Cafe Punjabi,Restaurant_267,Chinese,4.2
9997,9998,11-11-2023,979.44,522,User_522,Bangalore,Gold,420,Ruchi Tiffins Chinese,Restaurant_420,Italian,4.0
9998,9999,08-09-2023,1105.93,319,User_319,Bangalore,Gold,492,Swagath Kitchen North Indian,Restaurant_492,Italian,4.0


## Save to CSV

In [48]:
final_dataset.to_csv("final_food_delivery_data.csv", index=False)